In [1]:
#@ Initializing necessary Dependencies:
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn

In [2]:
x=[[1, 2], [3, 4], [4, 5], [6, 7]]
y=[[8], [9], [10], [11]]


In [3]:
X=torch.tensor(x).float()
Y=torch.tensor(y).float()

In [4]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
X=X.to(device)
Y=Y.to(device)

In [6]:
class MyDataSet(Dataset):
  def __init__(self, x, y):
    self.x=torch.tensor(x).float()
    self.y=torch.tensor(y).float()

  #@ for calculating the number of datapoints:
  def __len__(self):
    return len(self.x)

  #@ For fetching speoific row:
  def __getitem__(self, ix):     # 'ix' represents the index of the row
    return self.x[ix], self.y[ix]

In [7]:
ds=MyDataSet(X, y)

<ipython-input-6-712895d14ae1>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x=torch.tensor(x).float()


In [8]:
dl=DataLoader(ds, batch_size=2, shuffle=True)

In [9]:
for x, y in dl:
  print(x, y)

tensor([[1., 2.],
        [4., 5.]], device='cuda:0') tensor([[ 8.],
        [10.]])
tensor([[6., 7.],
        [3., 4.]], device='cuda:0') tensor([[11.],
        [ 9.]])


In [10]:
# #@  Creating Model:

# class NeuralNet(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.inputlayer=nn.Linear(2, 8)
#     self.hiddenlayer=nn.ReLU()
#     self.outputlayer=nn.Linear(8, 1)

#   def forward(self, x):
#     hidden1=self.inputlayer(x)
#     hidden2=self.hiddenlayer(x)
#     output=self.outputlayer(hidden2)
#     return output, hidden2



In [11]:
model=nn.Sequential(
        nn.Linear(2, 8),
        nn.ReLU(),
        nn.Linear(8, 1)
).to(device)

In [12]:
%pip install torch_summary
from torchsummary import summary

In [13]:
summary(model, torch.zeros(1,2));

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 8]                   24
├─ReLU: 1-2                              [-1, 8]                   --
├─Linear: 1-3                            [-1, 1]                   9
Total params: 33
Trainable params: 33
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [15]:
loss_function=nn.MSELoss()
from torch.optim import SGD
opt=SGD(model.parameters(), lr=0.01)

In [16]:
import time
Loss_history=[]
start=time.time()
for _ in range(50):
   for ix, iy in dl:
    ix, iy= ix.to(device), iy.to(device)
    opt.zero_grad()
    loss_value=loss_function(model(ix), iy)
    loss_value.backward()
    opt.step()
    Loss_history.append(loss_value.item())
end=time.time()
print(end - start)

0.4945359230041504


In [17]:
#@ Validation set:

valid=[[8, 9], [10, 11], [1.5, 3.6]]

In [18]:
model(torch.tensor(valid).float().to(device))

tensor([[14.3655],
        [16.5258],
        [ 9.3130]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [20]:
#@ Saving the model:
torch.save(model.to('cpu').state_dict(), 'mymodel.pth')

In [22]:
#@ Loading the model:
   # creating an empty model as loading model requires to initialize
   # the model with random weights first and then load
   # the weights from state_dict

model=nn.Sequential(
               nn.Linear(2, 8),
               nn.ReLU(),
               nn.Linear(8, 1)
).to(device)

In [24]:
state_dict=torch.load('mymodel.pth')

In [26]:
model.load_state_dict(state_dict)
model.to(device)
model(torch.tensor(valid).float().to(device))

tensor([[14.3655],
        [16.5258],
        [ 9.3130]], device='cuda:0', grad_fn=<AddmmBackward0>)